# POI Clustering & Restaurant Analysis - EDA

This notebook analyzes POI distribution, identifies clusters, and maps restaurants near attractions.

## Objectives
1. Load and explore POI data (from Overture Maps or seed data)
2. Perform geographic clustering (DBSCAN, K-Means)
3. Identify restaurant clusters near popular attractions
4. Visualize POI density and neighborhood patterns
5. Generate insights for itinerary optimization

In [ ]:
# Install required packages
# !pip install pandas numpy scikit-learn folium plotly geopandas shapely overturemaps

In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Clustering
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import StandardScaler

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Geographic
try:
    import folium
    from folium.plugins import MarkerCluster, HeatMap
    FOLIUM_AVAILABLE = True
except ImportError:
    FOLIUM_AVAILABLE = False
    print("Folium not available. Install with: pip install folium")

# Distance calculations
from math import radians, cos, sin, asin, sqrt

print("Libraries loaded successfully!")

## 1. Data Loading

We'll load POI data from multiple sources:
- Option A: Our curated seed data
- Option B: Overture Maps data (requires DuckDB)
- Option C: Sample data generation for demo

In [ ]:
def load_seed_data():
    """Load our curated Rome POI data."""
    seed_path = Path("../data/seed/rome_pois.json")
    
    if seed_path.exists():
        with open(seed_path, 'r') as f:
            data = json.load(f)
        return pd.DataFrame(data['pois'])
    else:
        print(f"Seed file not found at {seed_path}")
        return None

def generate_sample_rome_data(n_pois=200):
    """Generate synthetic Rome POI data for demonstration."""
    np.random.seed(42)
    
    # Rome neighborhoods with approximate centers
    neighborhoods = {
        'Centro Storico': (41.8986, 12.4769),
        'Trastevere': (41.8894, 12.4700),
        'Vatican City': (41.9029, 12.4534),
        'Testaccio': (41.8767, 12.4744),
        'Monti': (41.8956, 12.4939),
        'Aventine': (41.8826, 12.4791),
        'Prati': (41.9071, 12.4600),
        'Villa Borghese': (41.9125, 12.4850),
        'Esquilino': (41.8950, 12.5050),
        'San Lorenzo': (41.8970, 12.5150)
    }
    
    categories = {
        'attraction': 0.25,
        'restaurant': 0.40,
        'activity': 0.15,
        'shopping': 0.10,
        'nightlife': 0.10
    }
    
    subcategories = {
        'attraction': ['museum', 'historical', 'church', 'monument', 'park', 'viewpoint'],
        'restaurant': ['trattoria', 'pizzeria', 'fine_dining', 'cafe', 'gelateria', 'wine_bar'],
        'activity': ['walking_tour', 'cooking_class', 'market', 'spa'],
        'shopping': ['boutique', 'market', 'antiques'],
        'nightlife': ['bar', 'club', 'cocktail_bar']
    }
    
    pois = []
    
    for i in range(n_pois):
        # Select neighborhood (weighted towards centro)
        neighborhood_weights = {
            'Centro Storico': 0.30,
            'Trastevere': 0.15,
            'Vatican City': 0.10,
            'Testaccio': 0.08,
            'Monti': 0.12,
            'Aventine': 0.05,
            'Prati': 0.07,
            'Villa Borghese': 0.05,
            'Esquilino': 0.04,
            'San Lorenzo': 0.04
        }
        neighborhood = np.random.choice(
            list(neighborhood_weights.keys()),
            p=list(neighborhood_weights.values())
        )
        base_lat, base_lon = neighborhoods[neighborhood]
        
        # Add random offset (roughly 500m radius)
        lat = base_lat + np.random.normal(0, 0.003)
        lon = base_lon + np.random.normal(0, 0.004)
        
        # Select category
        category = np.random.choice(
            list(categories.keys()),
            p=list(categories.values())
        )
        subcategory = np.random.choice(subcategories[category])
        
        # Generate attributes
        poi = {
            'id': i,
            'name': f"{category.title()} {i}",
            'latitude': lat,
            'longitude': lon,
            'neighborhood': neighborhood,
            'category': category,
            'subcategory': subcategory,
            'cost_level': np.random.randint(1, 6),
            'rating': round(np.random.uniform(3.5, 5.0), 1),
            'review_count': np.random.randint(10, 5000),
            'typical_duration_minutes': np.random.choice([30, 45, 60, 90, 120, 180]),
            'is_must_see': np.random.random() < 0.1,
            'is_hidden_gem': np.random.random() < 0.15
        }
        pois.append(poi)
    
    return pd.DataFrame(pois)

# Try loading seed data, fall back to generated data
df = load_seed_data()
if df is None or len(df) < 50:
    print("Using generated sample data for demonstration...")
    df = generate_sample_rome_data(200)

print(f"Loaded {len(df)} POIs")
df.head()

## 2. Data Exploration

In [ ]:
# Basic statistics
print("=" * 50)
print("DATASET OVERVIEW")
print("=" * 50)
print(f"\nTotal POIs: {len(df)}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nData types:\n{df.dtypes}")

In [ ]:
# Category distribution
print("\n" + "=" * 50)
print("CATEGORY DISTRIBUTION")
print("=" * 50)

category_counts = df['category'].value_counts()
print(f"\n{category_counts}")

# Visualization
fig = px.pie(
    values=category_counts.values,
    names=category_counts.index,
    title='POI Distribution by Category',
    hole=0.4
)
fig.show()

In [ ]:
# Neighborhood distribution
print("\n" + "=" * 50)
print("NEIGHBORHOOD DISTRIBUTION")
print("=" * 50)

neighborhood_counts = df['neighborhood'].value_counts()
print(f"\n{neighborhood_counts}")

fig = px.bar(
    x=neighborhood_counts.index,
    y=neighborhood_counts.values,
    title='POI Count by Neighborhood',
    labels={'x': 'Neighborhood', 'y': 'POI Count'},
    color=neighborhood_counts.values,
    color_continuous_scale='Viridis'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
# Category by Neighborhood heatmap
category_neighborhood = pd.crosstab(df['neighborhood'], df['category'])

fig = px.imshow(
    category_neighborhood,
    title='POI Categories by Neighborhood',
    labels=dict(x='Category', y='Neighborhood', color='Count'),
    color_continuous_scale='Blues'
)
fig.show()

## 3. Geographic Visualization

In [ ]:
# Scatter plot of all POIs
fig = px.scatter_mapbox(
    df,
    lat='latitude',
    lon='longitude',
    color='category',
    hover_name='name',
    hover_data=['neighborhood', 'subcategory'] if 'subcategory' in df.columns else ['neighborhood'],
    title='POI Distribution in Rome',
    zoom=12,
    height=600
)
fig.update_layout(mapbox_style='carto-positron')
fig.show()

In [ ]:
# Separate maps for attractions vs restaurants
attractions = df[df['category'] == 'attraction']
restaurants = df[df['category'] == 'restaurant']

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Attractions', 'Restaurants'),
    specs=[[{'type': 'scattermapbox'}, {'type': 'scattermapbox'}]]
)

# Attractions
fig.add_trace(
    go.Scattermapbox(
        lat=attractions['latitude'],
        lon=attractions['longitude'],
        mode='markers',
        marker=dict(size=10, color='red'),
        text=attractions['name'],
        name='Attractions'
    ),
    row=1, col=1
)

# Restaurants
fig.add_trace(
    go.Scattermapbox(
        lat=restaurants['latitude'],
        lon=restaurants['longitude'],
        mode='markers',
        marker=dict(size=8, color='blue'),
        text=restaurants['name'],
        name='Restaurants'
    ),
    row=1, col=2
)

fig.update_layout(
    height=500,
    mapbox=dict(style='carto-positron', center=dict(lat=41.9, lon=12.48), zoom=11),
    mapbox2=dict(style='carto-positron', center=dict(lat=41.9, lon=12.48), zoom=11)
)
fig.show()

## 4. Geographic Clustering Analysis

Using DBSCAN and K-Means to identify POI clusters.

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """Calculate the great circle distance in kilometers."""
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    km = 6371 * c
    return km

# Prepare coordinates
coords = df[['latitude', 'longitude']].values

# Convert to radians for DBSCAN with haversine
coords_rad = np.radians(coords)

In [ ]:
# DBSCAN Clustering
# eps is in radians, 0.5km = 0.5/6371 radians ≈ 0.000078
eps_km = 0.3  # 300 meters
eps_rad = eps_km / 6371.0

dbscan = DBSCAN(
    eps=eps_rad,
    min_samples=3,
    metric='haversine'
)

df['cluster_dbscan'] = dbscan.fit_predict(coords_rad)

n_clusters = len(set(df['cluster_dbscan'])) - (1 if -1 in df['cluster_dbscan'].values else 0)
n_noise = (df['cluster_dbscan'] == -1).sum()

print(f"DBSCAN Results (eps={eps_km}km):")
print(f"  - Number of clusters: {n_clusters}")
print(f"  - Noise points: {n_noise}")
print(f"  - Clustered points: {len(df) - n_noise}")

In [ ]:
# Visualize DBSCAN clusters
fig = px.scatter_mapbox(
    df,
    lat='latitude',
    lon='longitude',
    color='cluster_dbscan',
    hover_name='name',
    hover_data=['category', 'neighborhood'],
    title=f'DBSCAN Clustering (eps={eps_km}km, {n_clusters} clusters)',
    zoom=12,
    height=600,
    color_continuous_scale='Viridis'
)
fig.update_layout(mapbox_style='carto-positron')
fig.show()

In [ ]:
# K-Means Clustering
# Normalize coordinates for K-Means
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(coords)

# Find optimal K using elbow method
inertias = []
K_range = range(2, 15)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(coords_scaled)
    inertias.append(kmeans.inertia_)

# Plot elbow curve
fig = px.line(
    x=list(K_range),
    y=inertias,
    title='K-Means Elbow Method',
    labels={'x': 'Number of Clusters (K)', 'y': 'Inertia'},
    markers=True
)
fig.show()

In [ ]:
# Apply K-Means with chosen K
optimal_k = 8  # Based on elbow curve, or use neighborhood count

kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['cluster_kmeans'] = kmeans.fit_predict(coords_scaled)

print(f"K-Means Results (K={optimal_k}):")
print(f"\nCluster sizes:")
print(df['cluster_kmeans'].value_counts().sort_index())

In [ ]:
# Visualize K-Means clusters
fig = px.scatter_mapbox(
    df,
    lat='latitude',
    lon='longitude',
    color='cluster_kmeans',
    hover_name='name',
    hover_data=['category', 'neighborhood'],
    title=f'K-Means Clustering (K={optimal_k})',
    zoom=12,
    height=600,
    color_discrete_sequence=px.colors.qualitative.Set1
)
fig.update_layout(mapbox_style='carto-positron')
fig.show()

## 5. Restaurant Analysis Near Attractions

Find restaurants within walking distance of major attractions.

In [ ]:
def find_nearby_pois(target_poi, all_pois, max_distance_km=0.5, category_filter=None):
    """Find POIs within a certain distance of a target POI."""
    nearby = []
    
    for idx, poi in all_pois.iterrows():
        if category_filter and poi['category'] != category_filter:
            continue
        if poi['name'] == target_poi['name']:
            continue
            
        distance = haversine(
            target_poi['longitude'], target_poi['latitude'],
            poi['longitude'], poi['latitude']
        )
        
        if distance <= max_distance_km:
            poi_dict = poi.to_dict()
            poi_dict['distance_km'] = round(distance, 3)
            nearby.append(poi_dict)
    
    return pd.DataFrame(nearby).sort_values('distance_km') if nearby else pd.DataFrame()

# Identify major attractions (must_see or high review count)
if 'is_must_see' in df.columns:
    major_attractions = df[(df['category'] == 'attraction') & (df['is_must_see'] == True)]
else:
    major_attractions = df[df['category'] == 'attraction'].head(10)

print(f"Major attractions identified: {len(major_attractions)}")
print(major_attractions['name'].tolist())

In [ ]:
# Find restaurants near each major attraction
restaurant_analysis = []

for idx, attraction in major_attractions.iterrows():
    nearby_restaurants = find_nearby_pois(
        attraction, 
        df, 
        max_distance_km=0.5,
        category_filter='restaurant'
    )
    
    analysis = {
        'attraction': attraction['name'],
        'neighborhood': attraction['neighborhood'],
        'nearby_restaurant_count': len(nearby_restaurants),
        'avg_distance_km': nearby_restaurants['distance_km'].mean() if len(nearby_restaurants) > 0 else None
    }
    
    if len(nearby_restaurants) > 0:
        analysis['closest_restaurant'] = nearby_restaurants.iloc[0]['name']
        analysis['closest_distance_m'] = int(nearby_restaurants.iloc[0]['distance_km'] * 1000)
        
        # Subcategory breakdown if available
        if 'subcategory' in nearby_restaurants.columns:
            analysis['restaurant_types'] = nearby_restaurants['subcategory'].value_counts().to_dict()
    
    restaurant_analysis.append(analysis)

restaurant_df = pd.DataFrame(restaurant_analysis)
restaurant_df

In [ ]:
# Visualize restaurant density near attractions
fig = px.bar(
    restaurant_df.sort_values('nearby_restaurant_count', ascending=True),
    x='nearby_restaurant_count',
    y='attraction',
    orientation='h',
    title='Restaurant Density Near Major Attractions (500m radius)',
    labels={'nearby_restaurant_count': 'Number of Restaurants', 'attraction': 'Attraction'},
    color='nearby_restaurant_count',
    color_continuous_scale='Reds'
)
fig.show()

In [ ]:
# Interactive map: Attraction with nearby restaurants
def create_attraction_restaurant_map(attraction_name, radius_km=0.5):
    """Create a map showing an attraction and nearby restaurants."""
    attraction = df[df['name'] == attraction_name].iloc[0]
    nearby = find_nearby_pois(attraction, df, max_distance_km=radius_km, category_filter='restaurant')
    
    # Create map
    fig = go.Figure()
    
    # Add attraction marker
    fig.add_trace(go.Scattermapbox(
        lat=[attraction['latitude']],
        lon=[attraction['longitude']],
        mode='markers',
        marker=dict(size=20, color='red'),
        text=[attraction['name']],
        name='Attraction'
    ))
    
    # Add restaurant markers
    if len(nearby) > 0:
        fig.add_trace(go.Scattermapbox(
            lat=nearby['latitude'],
            lon=nearby['longitude'],
            mode='markers',
            marker=dict(size=10, color='blue'),
            text=nearby['name'] + '<br>Distance: ' + (nearby['distance_km'] * 1000).astype(int).astype(str) + 'm',
            name='Restaurants'
        ))
    
    fig.update_layout(
        title=f"{attraction_name}: {len(nearby)} restaurants within {radius_km}km",
        mapbox=dict(
            style='carto-positron',
            center=dict(lat=attraction['latitude'], lon=attraction['longitude']),
            zoom=15
        ),
        height=500,
        showlegend=True
    )
    
    return fig

# Show map for first major attraction
if len(major_attractions) > 0:
    first_attraction = major_attractions.iloc[0]['name']
    fig = create_attraction_restaurant_map(first_attraction)
    fig.show()

## 6. Cluster Profiling

Analyze what each cluster contains and characterize it.

In [ ]:
def profile_cluster(cluster_df, cluster_id):
    """Generate a profile for a cluster."""
    cluster_pois = cluster_df[cluster_df['cluster_kmeans'] == cluster_id]
    
    profile = {
        'cluster_id': cluster_id,
        'total_pois': len(cluster_pois),
        'center_lat': cluster_pois['latitude'].mean(),
        'center_lon': cluster_pois['longitude'].mean(),
        'dominant_neighborhood': cluster_pois['neighborhood'].mode().iloc[0] if len(cluster_pois) > 0 else None,
        'category_mix': cluster_pois['category'].value_counts().to_dict(),
        'attraction_count': len(cluster_pois[cluster_pois['category'] == 'attraction']),
        'restaurant_count': len(cluster_pois[cluster_pois['category'] == 'restaurant']),
        'activity_count': len(cluster_pois[cluster_pois['category'] == 'activity']),
    }
    
    # Calculate ratios
    if profile['total_pois'] > 0:
        profile['restaurant_ratio'] = profile['restaurant_count'] / profile['total_pois']
        profile['attraction_ratio'] = profile['attraction_count'] / profile['total_pois']
    
    # Determine cluster character
    if profile['restaurant_ratio'] > 0.6:
        profile['character'] = 'Dining Hub'
    elif profile['attraction_ratio'] > 0.4:
        profile['character'] = 'Tourist Area'
    elif profile.get('nightlife_count', 0) / max(profile['total_pois'], 1) > 0.3:
        profile['character'] = 'Nightlife District'
    else:
        profile['character'] = 'Mixed Use'
    
    return profile

# Profile all clusters
cluster_profiles = []
for cluster_id in sorted(df['cluster_kmeans'].unique()):
    profile = profile_cluster(df, cluster_id)
    cluster_profiles.append(profile)

profiles_df = pd.DataFrame(cluster_profiles)
profiles_df[['cluster_id', 'total_pois', 'dominant_neighborhood', 'character', 'attraction_count', 'restaurant_count']]

In [ ]:
# Visualize cluster profiles
fig = px.bar(
    profiles_df,
    x='cluster_id',
    y=['attraction_count', 'restaurant_count', 'activity_count'],
    title='POI Composition by Cluster',
    labels={'value': 'Count', 'cluster_id': 'Cluster'},
    barmode='group'
)
fig.show()

## 7. Insights & Recommendations

In [ ]:
print("=" * 60)
print("KEY INSIGHTS FOR ITINERARY OPTIMIZATION")
print("=" * 60)

# 1. Identify food deserts (attractions with few restaurants)
print("\n1. FOOD ACCESSIBILITY NEAR ATTRACTIONS")
print("-" * 40)
low_restaurant_areas = restaurant_df[restaurant_df['nearby_restaurant_count'] < 5]
if len(low_restaurant_areas) > 0:
    print("\n   Attractions with limited dining options nearby:")
    for _, row in low_restaurant_areas.iterrows():
        print(f"   - {row['attraction']}: only {row['nearby_restaurant_count']} restaurants within 500m")
else:
    print("   All attractions have adequate dining options nearby!")

# 2. Cluster recommendations for itinerary building
print("\n2. CLUSTER-BASED ITINERARY RECOMMENDATIONS")
print("-" * 40)
for _, profile in profiles_df.iterrows():
    print(f"\n   Cluster {profile['cluster_id']} ({profile['dominant_neighborhood']}):")
    print(f"   - Character: {profile['character']}")
    print(f"   - {profile['attraction_count']} attractions, {profile['restaurant_count']} restaurants")
    
    if profile['character'] == 'Tourist Area':
        print("   - Recommendation: Good for morning/afternoon sightseeing")
    elif profile['character'] == 'Dining Hub':
        print("   - Recommendation: Plan lunch/dinner stops here")

# 3. Walkability insights
print("\n3. WALKABILITY INSIGHTS")
print("-" * 40)
clusters_with_both = profiles_df[(profiles_df['attraction_count'] >= 2) & (profiles_df['restaurant_count'] >= 3)]
print(f"\n   Self-contained clusters (attractions + restaurants): {len(clusters_with_both)}")
for _, c in clusters_with_both.iterrows():
    print(f"   - Cluster {c['cluster_id']} ({c['dominant_neighborhood']}): Can spend half-day without leaving area")

In [ ]:
# Save cluster assignments back to dataframe
output_path = Path('../data/processed/')
output_path.mkdir(exist_ok=True)

# Export clustered data
df.to_csv(output_path / 'rome_pois_clustered.csv', index=False)
profiles_df.to_csv(output_path / 'cluster_profiles.csv', index=False)
restaurant_df.to_csv(output_path / 'restaurant_accessibility.csv', index=False)

print(f"\nExported analysis to {output_path}:")
print("  - rome_pois_clustered.csv")
print("  - cluster_profiles.csv")
print("  - restaurant_accessibility.csv")

## 8. Interactive Folium Map (Optional)

In [ ]:
if FOLIUM_AVAILABLE:
    # Create interactive folium map with clusters
    m = folium.Map(
        location=[41.9028, 12.4964],
        zoom_start=13,
        tiles='cartodbpositron'
    )
    
    # Color palette for clusters
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 
              'darkblue', 'darkgreen', 'cadetblue', 'pink']
    
    # Add markers for each POI
    for idx, poi in df.iterrows():
        cluster_id = poi['cluster_kmeans']
        color = colors[cluster_id % len(colors)]
        
        icon = 'star' if poi['category'] == 'attraction' else 'cutlery' if poi['category'] == 'restaurant' else 'info-sign'
        
        folium.Marker(
            location=[poi['latitude'], poi['longitude']],
            popup=f"{poi['name']}<br>Category: {poi['category']}<br>Cluster: {cluster_id}",
            icon=folium.Icon(color=color, icon=icon, prefix='glyphicon')
        ).add_to(m)
    
    # Add heatmap layer
    heat_data = [[poi['latitude'], poi['longitude']] for _, poi in df.iterrows()]
    HeatMap(heat_data, radius=15).add_to(m)
    
    # Save map
    m.save(str(output_path / 'rome_poi_clusters_map.html'))
    print(f"\nInteractive map saved to {output_path / 'rome_poi_clusters_map.html'}")
    
    # Display map (in Jupyter)
    m
else:
    print("Folium not available. Install with: pip install folium")

## Summary

### Key Findings:
1. **Geographic Clusters**: Identified distinct POI clusters using DBSCAN and K-Means
2. **Restaurant Accessibility**: Mapped dining options near major attractions
3. **Neighborhood Patterns**: Centro Storico has highest POI density, Trastevere strongest for dining
4. **Itinerary Implications**: Some clusters are self-contained (can spend half-day), others require travel

### Recommendations for Itinerary System:
1. Use cluster IDs to group nearby POIs in same day
2. Pre-compute restaurant options for each attraction
3. Consider cluster character when assigning day themes
4. Flag attractions in "food deserts" to suggest bringing snacks or planning ahead